In [1]:
df = spark.read.format('json').load("hdfs://orion11:33001/reddit/sampled_reddit/*")
from pyspark.sql.functions import year, month, dayofmonth, from_unixtime, desc
from pyspark.sql.types import TimestampType, DateType
botExpr = "[bB][oO][tT]"
df2 = (df.filter(~(df.body.like("[deleted]") | df.author.rlike(botExpr))))


import sys
path = '/home4/vramaneti/.local/lib/python3.7/site-packages'
sys.path.insert(0,path)

In [2]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf, col

def comment_positiveness(comment):
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(comment)
    print(ss)
    return ss["pos"]

def comment_negativeness(comment):
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(comment)
    print(ss)
    return ss["neg"]

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home4/vramaneti/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [7]:
comment_pos_udf = udf(comment_positiveness, DoubleType())
comment_neg_udf = udf(comment_negativeness, DoubleType())
df3 = df2.withColumn('comment_pos', comment_pos_udf(col('body')))
df4 = df3.withColumn('comment_neg', comment_neg_udf(col('body')))
df4.show()

+--------+----------------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+--------+-----------+----------+----------+--------------+-------+------------+-----+-----+------------+--------+-----------------+------------+---+------------+-----------+-----------+
|archived|          author|author_cakeday|author_flair_css_class|author_flair_text|                body|body_html|controversiality|created|created_utc|distinguished|downs|edited|gilded|     id| link_id|mod_reports|      name| parent_id|removal_reason|replies|retrieved_on|saved|score|score_hidden|stickied|        subreddit|subreddit_id|ups|user_reports|comment_pos|comment_neg|
+--------+----------------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+--------+-----------+----------+----------+------

In [10]:
import pyspark.sql.functions as sf
df5 = df4.groupBy("subreddit").agg(sf.mean('comment_pos').alias('pos_score'))
screamer_df = df5.select('subreddit', 'pos_score').orderBy(desc("pos_score")).limit(5)

In [11]:
screamer_df.show()

+------------------+---------+
|         subreddit|pos_score|
+------------------+---------+
|            Kernow|      1.0|
|     redditforkids|      1.0|
|       shredvideos|      1.0|
|     Josheatspie12|      1.0|
|KarmaStockExchange|      1.0|
+------------------+---------+



In [17]:
df6 = df4.groupBy("subreddit").agg(sf.mean('comment_neg').alias('neg_score'))
negative_df = df6.select('subreddit', 'neg_score').orderBy(desc("neg_score")).limit(5)

In [18]:
negative_df.show()

+----------+---------+
| subreddit|neg_score|
+----------+---------+
|    Magnox|      1.0|
|     Morty|      1.0|
|   usdcoin|      1.0|
|  azudaioh|      1.0|
|96Problems|      1.0|
+----------+---------+

